In [ ]:
# try to optimize oppy strategy of composer/oppy/oppy.py

In [4]:
from basebot22.basebot import BaseBot
from datetime import datetime, timedelta
import pandas as pd
bot = BaseBot("testbot",) #  backendurl = "http://tradingbot-baseimage-service:8000"
symbols = ['NTSX', 'GLD', 'UUP', 'DBMF', 'VDE', 'VNQ', 'VHT', 'VFH', 'VOX', 'VPU', 
        'VAW', 'VGT', 'VIS', 'VDC', 'VCR', 'VLUE', 'FNDX', 'VTV', 'RWL', 'DBA', 'SHV', 
        'DBB', 'DBO', 'URA', 'WOOD', 'DBE']

In [2]:
datas = dict()
for ticker in symbols:
    datas[ticker] = bot.getData(ticker, start_date = datetime.now() - timedelta(days=int(365*3)))

In [6]:
# helper
def effcore(budget: float):
    return {"NTSX": 1. * budget} # bc we want to return this 

def calculateCurrentVolatility(df: pd.DataFrame, range: int = 45):
    df["daily_returns"] = df["adj_close"].pct_change()
    # 45 bc we need 45 d volatility
    df["daily_volatility"] = df["daily_returns"].rolling(range).std()
    return df["daily_volatility"][-1]

def inverseVolatilityWeights(datas, tickers: list, range: int = 45):
    crntVolatilities = dict()
    for ticker in tickers:
        crntVolatilities[ticker] = calculateCurrentVolatility(datas[ticker], range)
    # create weights according to inverse volatilities (lower volatilities get higher weights)
    print("volatilities: " + str(crntVolatilities))
    for ticker, volatility in crntVolatilities.items():
        crntVolatilities[ticker] = 1 / volatility
    weights = dict()
    for ticker, volatility in crntVolatilities.items():
        weights[ticker] = crntVolatilities[ticker] / sum(crntVolatilities.values())
    print("weights: " + str(weights))
    return weights

def volhedge(datas, budget: float):
    weights = inverseVolatilityWeights(datas, ["GLD", "UUP", "DBMF"], 30)
    return {k: v * budget for k, v in weights.items()}

def sectormom(datas, budget: float, range: int = 200):
    tickers = ["VDE", "VNQ", "VHT", "VFH", "VOX", "VPU", "VAW", "VGT", "VIS", "VDC", "VCR"]
    cumret = dict()
    for ticker in tickers:
        df = datas[ticker]
        df["cumulative_return"] = (df["close"] / df["close"].shift(range)) - 1
        cumret[ticker] = df["cumulative_return"].iloc[-1]
        assert cumret[ticker] is not None, f"cumret is None for {ticker}"
    cumret = {k: v for k, v in sorted(cumret.items(), key=lambda item: item[1], reverse=True)}
    print("sectormom cumret: ", cumret)
    biggestTwo = list(cumret.keys())[:2]
    return {biggestTwo[0]: 0.5 * budget, biggestTwo[1]: 0.5 * budget}

def largecapval(datas, budget: float):
    weights = inverseVolatilityWeights(datas, ["VLUE", "FNDX", "VTV", "RWL"], 21)
    return {k: v * budget for k, v in weights.items()}

def __buyHelper(tobuy: dict, cumret: dict, ticker: str):
    if cumret[ticker] > cumret["SHV"]:
        if ticker not in tobuy:
            tobuy[ticker] = 0
        tobuy[ticker] +=  1
    else:
        if "SHV" not in tobuy:
            tobuy["SHV"] = 0
        tobuy["SHV"] += 1
    return tobuy, cumret

def commodmom(datas, budget: float, range: int = 42):
    symbols = ['DBA', 'SHV', 'DBB', 'DBO', 'URA', 'WOOD', 'DBE', 'GLD']
    tobuy = dict()
    cumret = dict()
    # get several cum returns
    for ticker in symbols:
        df = datas[ticker]
        df["cumulative_return"] = (df["close"] / df["close"].shift(range)) - 1
        cumret[ticker] = df["cumulative_return"].iloc[-1]
    ## logics
    # agriculture 
    tobuy, cumret = __buyHelper(tobuy, cumret, "DBA")
    # base metals
    tobuy, cumret = __buyHelper(tobuy, cumret, "DBB")
    # oil
    tobuy, cumret = __buyHelper(tobuy, cumret, "DBO")
    # uranium
    tobuy, cumret = __buyHelper(tobuy, cumret, "URA")
    # timber
    tobuy, cumret = __buyHelper(tobuy, cumret, "WOOD")
    # gold
    tobuy, cumret = __buyHelper(tobuy, cumret, "GLD")
    # energy
    tobuy, cumret = __buyHelper(tobuy, cumret, "DBE")
    
    # convert the counts to weights
    tobuy = {k: v / sum(tobuy.values()) for k, v in tobuy.items()}
    print("commodmom tobuy: ", tobuy)
    return {k: v * budget for k, v in tobuy.items()}

In [9]:
import warnings
warnings.filterwarnings("ignore")
# yolo

In [10]:
# opus default
# weights
# 15% eff core
# 25% vol hedge
# 25% sector momentum
# 15% large cap val
# 20% commodity momentum
portfolioWorth = 10000
datas_train = dict()
datas_test = dict()
for ticker in symbols:
    datas_train[ticker] = datas[ticker][:int(len(datas[ticker]) * 0.4)]
    datas_test[ticker] = datas[ticker][int(len(datas[ticker]) * 0.4):]

effcoreweights = effcore(0.15 * 1)
volhedgeweights = volhedge(datas_train, 0.25 * 1)
sectormomweights = sectormom(datas_train, 0.25 * 1)
largecapvalweights = largecapval(datas_train, 0.15 * 1)
commodmomweights = commodmom(datas_train, 0.20 * 1)
# merge
allWeights = {**effcoreweights, **volhedgeweights, **sectormomweights, **largecapvalweights, **commodmomweights}
allWeights

volatilities: {'GLD': 0.010408978271971061, 'UUP': 0.00359210335454936, 'DBMF': 0.010653434338288716}
weights: {'GLD': 0.20513692149609364, 'UUP': 0.5944332742897225, 'DBMF': 0.20042980421418397}
sectormom cumret:  {'VCR': 0.5568588146235256, 'VFH': 0.48746092625795323, 'VIS': 0.4616286348660352, 'VOX': 0.4460185131130525, 'VAW': 0.4374789710806197, 'VDE': 0.39419903125846334, 'VGT': 0.3917557761324082, 'VNQ': 0.19591887006529274, 'VHT': 0.16041099241266754, 'VDC': 0.15430418389460265, 'VPU': 0.060781562909646425}
volatilities: {'VLUE': 0.012803192797389071, 'FNDX': 0.011627666820547724, 'VTV': 0.010378189492168786, 'RWL': 0.010363294473252257}
weights: {'VLUE': 0.21880893413986402, 'FNDX': 0.2409299314143853, 'VTV': 0.2699365791786532, 'RWL': 0.27032455526709753}
commodmom tobuy:  {'DBA': 0.14285714285714285, 'DBB': 0.14285714285714285, 'DBO': 0.14285714285714285, 'URA': 0.14285714285714285, 'WOOD': 0.14285714285714285, 'SHV': 0.14285714285714285, 'DBE': 0.14285714285714285}


{'NTSX': 0.15,
 'GLD': 0.05128423037402341,
 'UUP': 0.14860831857243062,
 'DBMF': 0.05010745105354599,
 'VCR': 0.125,
 'VFH': 0.125,
 'VLUE': 0.0328213401209796,
 'FNDX': 0.036139489712157795,
 'VTV': 0.04049048687679798,
 'RWL': 0.04054868329006463,
 'DBA': 0.02857142857142857,
 'DBB': 0.02857142857142857,
 'DBO': 0.02857142857142857,
 'URA': 0.02857142857142857,
 'WOOD': 0.02857142857142857,
 'SHV': 0.02857142857142857,
 'DBE': 0.02857142857142857}

In [12]:
assert sum(allWeights.values()) == 1, "weights don't sum to 1"

In [17]:
def buyAccordingToWeights(weights:dict, money: float, datas):
    portfolio = dict()
    for ticker, weight in weights.items():
        crntPrice = datas[ticker]["close"].iloc[-1]
        howMany = weight * money / crntPrice
        portfolio[ticker] = howMany
    return portfolio

In [19]:
# test it
startWeights = allWeights
portfolio = buyAccordingToWeights(startWeights, 10000, datas_train)
print(portfolio)


{'NTSX': 40.09623111783478, 'GLD': 3.158868398275525, 'UUP': 60.06803554861716, 'DBMF': 18.289393811605944, 'VCR': 4.223115785132488, 'VFH': 14.635287980376708, 'VLUE': 3.1763611283625766, 'FNDX': 6.945894783787304, 'VTV': 3.0847545765125917, 'RWL': 5.863025035130239, 'DBA': 16.458195603053014, 'DBB': 15.278838184890763, 'DBO': 25.442056755185668, 'URA': 14.165309104291444, 'WOOD': 3.302291918629917, 'SHV': 2.585415620226605, 'DBE': 20.540207681414284}


In [24]:
def calculatePortfolioWorth(portfolio: dict, datas: dict):
    totalWorth = 0
    positionsWorth = dict()
    for ticker, count in portfolio.items():
        w = count * datas[ticker]["close"].iloc[-1]
        positionsWorth[ticker] = w
        totalWorth += count * w
    return totalWorth, positionsWorth

In [28]:

from tqdm import tqdm
startMoney = 10000
worthTrack = []

for i in range(100, len(datas_test["NTSX"]), 20): # recalculate every month
    datas_subset = dict()
    for ticker in symbols:
        datas_subset[ticker] = datas_test[ticker][:i]
    # recalculate weights
    effcoreweights = effcore(0.15 * 1)
    volhedgeweights = volhedge(datas_subset, 0.25 * 1)
    sectormomweights = sectormom(datas_subset, 0.25 * 1)
    largecapvalweights = largecapval(datas_subset, 0.15 * 1)
    commodmomweights = commodmom(datas_subset, 0.20 * 1)
    # merge
    allWeights = {**effcoreweights, **volhedgeweights, **sectormomweights, **largecapvalweights, **commodmomweights}
    
    # calculate the diff to current portfolio
    totalWorth, positionsWorth = calculatePortfolioWorth(allWeights, datas_subset)
    worthTrack.append(totalWorth)
    # calculcate pct of position in portfolio
    positionsWorth = {k: v / totalWorth for k, v in positionsWorth.items()}
    
    diff = dict()
    tobuy = dict()
    for ticker, weight in allWeights.items():
        if ticker != "USD":
            if ticker not in positionsWorth:
                diff[ticker] = weight
            else:
                diff[ticker] = weight - positionsWorth.get(ticker, 0)
            # buy/sell according to diff
            if abs(diff[ticker]) > 0.02: # only buy if difference is bigger than 2 pct
                if diff[ticker] > 0:
                    # buy later
                    tobuy[ticker] = diff[ticker]
                else:
                    print("selling ", ticker, " for ", abs(diff[ticker]), " at ", datas_subset[ticker]["close"].iloc[-1], " for ", abs(diff[ticker]) * datas_subset[ticker]["close"].iloc[-1])
                    # sell now to get cash
                    if "USD" not in portfolio:
                        portfolio["USD"] = 0
                    portfolio["USD"] += abs(diff[ticker]) * datas_subset[ticker]["close"].iloc[-1]
                    portfolio[ticker] -= abs(diff[ticker])
    # buy according to tobuy
    for ticker, amount in tobuy.items():
        if "USD" not in portfolio:
            portfolio["USD"] = 0
        cost = amount * datas_subset[ticker]["close"].iloc[-1]
        if cost > portfolio["USD"]:
            # not enough money
            print("not enough money to buy ", ticker, " for ", amount, " at ", cost)
            continue
        portfolio["USD"] -= cost
        portfolio[ticker] += amount
        
print("done!")

    
    

volatilities: {'GLD': 0.006831989876012147, 'UUP': 0.002832049855810573, 'DBMF': 0.00902248797219956}
weights: {'GLD': 0.23983108476298684, 'UUP': 0.5785645120942859, 'DBMF': 0.18160440314272733}
sectormom cumret:  {'VDE': nan, 'VNQ': nan, 'VHT': nan, 'VFH': nan, 'VOX': nan, 'VPU': nan, 'VAW': nan, 'VGT': nan, 'VIS': nan, 'VDC': nan, 'VCR': nan}
volatilities: {'VLUE': 0.010273007782311962, 'FNDX': 0.009848092142582048, 'VTV': 0.008387777061776892, 'RWL': 0.008129065579389144}
weights: {'VLUE': 0.2206704616503569, 'FNDX': 0.23019173023965264, 'VTV': 0.270268195394818, 'RWL': 0.27886961271517247}
commodmom tobuy:  {'SHV': 0.8571428571428571, 'DBE': 0.14285714285714285}
selling  NTSX  for  0.5794433943221144  at  42.18000030517578  for  24.44092254933888
selling  GLD  for  1.0781269538236269  at  164.63999938964844  for  177.50282101948545
selling  UUP  for  0.2710854993975341  at  24.93000030517578  for  6.7581615827092545
selling  DBMF  for  0.09863164099912713  at  27.517000198364258  

KeyError: 'VDE'